In [3]:
# --- Chain-of-Thought Prompting Demo (safe, clean version) ---

import os
import re
from textwrap import dedent

# Detect API usage from environment only (never paste keys here)
USE_API = bool(os.getenv("OPENAI_API_KEY"))
FORCE_OFFLINE = False   # set True to force mocked demo even if a key exists

print(f"API mode: {'ON (using OpenAI API)' if USE_API and not FORCE_OFFLINE else 'OFFLINE (mocked demo)'}")

# ------------------------------------------------------------------
# Example problem (GSM8K-style reasoning task)
# ------------------------------------------------------------------
QUESTION = dedent("""
A rectangle has a length three times its width. Its perimeter is 64 units.
What is the area of the rectangle?
""").strip()

PROMPT_STANDARD = QUESTION
PROMPT_COT = QUESTION + "\nLet's think step by step."

# ------------------------------------------------------------------
# Helper: parse numeric answer from model text
# ------------------------------------------------------------------
def parse_final_number(text: str):
    """
    Extract the final numeric answer.
    1) Look for 'Answer: <number>'
    2) Otherwise, take the last integer/float in the text.
    Returns None if nothing found.
    """
    m = re.search(r'(?i)answer\s*[:=]\s*(-?\d+(\.\d+)?)', text)
    if m:
        return float(m.group(1))
    nums = re.findall(r'-?\d+\.?\d*', text)
    return float(nums[-1]) if nums else None


# ------------------------------------------------------------------
# Call the model (OpenAI API or mocked fallback)
# ------------------------------------------------------------------
def call_llm(prompt: str) -> str:
    if FORCE_OFFLINE or not USE_API:
        # Mocked outputs so the notebook always runs
        if "step by step" in prompt.lower():
            return dedent("""\
                Let's think step by step.
                Perimeter = 2(L + W) = 64 → L + W = 32.
                L = 3W, so 3W + W = 32 → 4W = 32 → W = 8.
                Then L = 3×8 = 24.
                Area = L × W = 24 × 8 = 192.
                Answer: 192
            """)
        else:
            return "Perimeter 64 so maybe area is 64. Answer: 64"
    else:
        # Real API call
        from openai import OpenAI
        client = OpenAI()
        resp = client.chat.completions.create(
            model="gpt-4o-mini",  # change if using another available model
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
        return resp.choices[0].message.content


# ------------------------------------------------------------------
# Run one trial (prints reasoning and parsed result)
# ------------------------------------------------------------------
def run_trial(label: str, prompt: str):
    print("=" * 80)
    print(label)
    print("-" * 80)
    output = call_llm(prompt)
    print(output.strip(), "\n")
    val = parse_final_number(output)
    print(f"Parsed numeric answer: {val}")
    return output, val


# ------------------------------------------------------------------
# Ground truth for this math problem
# ------------------------------------------------------------------
def ground_truth_area():
    # Perimeter P = 64, L = 3W
    # 2(L + W) = 64 -> L + W = 32 -> 3W + W = 32 -> 4W = 32 -> W = 8, L = 24
    # Area = 24 * 8 = 192
    return 192.0


# ------------------------------------------------------------------
# Main execution
# ------------------------------------------------------------------
gt = ground_truth_area()

std_text, std_val = run_trial("STANDARD PROMPT", PROMPT_STANDARD)
cot_text, cot_val = run_trial("CHAIN-OF-THOUGHT PROMPT", PROMPT_COT)

print("=" * 80)
print("RESULTS SUMMARY")
print("-" * 80)

def verdict(x):
    if x is None:
        return "no parse"
    return "correct" if abs(x - gt) < 1e-6 else "incorrect"

print(f"Ground truth: {gt}")
print(f"Standard: {std_val} -> {verdict(std_val)}")
print(f"CoT:      {cot_val} -> {verdict(cot_val)}")


API mode: ON (using OpenAI API)
STANDARD PROMPT
--------------------------------------------------------------------------------
Let the width of the rectangle be \( w \). According to the problem, the length \( l \) is three times the width, so we can express the length as:

\[
l = 3w
\]

The formula for the perimeter \( P \) of a rectangle is given by:

\[
P = 2l + 2w
\]

Substituting the expression for the length into the perimeter formula, we have:

\[
P = 2(3w) + 2w = 6w + 2w = 8w
\]

We know that the perimeter is 64 units, so we can set up the equation:

\[
8w = 64
\]

To find \( w \), we divide both sides by 8:

\[
w = \frac{64}{8} = 8
\]

Now that we have the width, we can find the length:

\[
l = 3w = 3 \times 8 = 24
\]

Next, we can calculate the area \( A \) of the rectangle using the formula:

\[
A = l \times w
\]

Substituting the values of \( l \) and \( w \):

\[
A = 24 \times 8 = 192
\]

Thus, the area of the rectangle is:

\[
\boxed{192} \text{ square units.}
\] 

Pars

In [2]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openai]━━━━ 1/2 [openai]
